In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
import tensorflow as tf
import datetime

In [3]:
# загрузим датасет
# здесь он упакован в бинарный файл посредством pickle

df = pd.read_pickle('df_pikabu_spam_posts.pd')

In [4]:
df.head

<bound method NDFrame.head of                                                title  \
0          [треб, помощник, работ, оффлайн, удаленк]   
1      [хоч, прода, аккаунт, pornhubpremium, реальн]   
2                             [нужн, помощ, кумерта]   
3                 [щенок, хаск, ищет, хозяин, платн]   
4                            [песик, пройд, опросик]   
...                                              ...   
11961                                [нич, жизн, уч]   
11962                               [девушк, работа]   
11963                                  [эт, неудобн]   
11964                           [репетиторск, истор]   
11965                             [лет, скетм, джон]   

                                                    text  bad  
0      [знает, процесс, регистрац, профсоюз, https, а...    1  
1      [появ, больш, количеств, аккаунт, сайт, реальн...    1  
2      [здравств, декабр, дедушк, упа, сво, квартир, ...    1  
3                     [мам, пап, хаск, по

In [5]:
# перемешаем датасет

df = df.sample(frac=1)

In [6]:
df['bad'].mean()

0.23579201934703747

In [7]:
# посчитаем количество уникальных слов, создав множество

words_set = set()

for row in df.itertuples():
    for word in row.title:
        words_set.add(word)
    for word in row.text:
        words_set.add(word)

print(f"Всего слов: {len(words_set)}")

Всего слов: 59073


In [8]:
# используем словарь для хранения кол-ва вхождений каждого слова
words_counter = {w: 0 for w in words_set}

for row in df.itertuples():
    for word in row.title:
        words_counter[word] += 1
    for word in row.text:
        words_counter[word] += 1

# преобразуем словарь в список и отсортируем его
words_list = list(words_counter.items())
words_list.sort(key=(lambda x: x[1]), reverse=True)

In [9]:
# наиболее популярные слова

words_list[:10]

[('эт', 11379),
 ('котор', 6957),
 ('сво', 4743),
 ('год', 4251),
 ('так', 4122),
 ('сам', 3757),
 ('одн', 3251),
 ('работ', 3207),
 ('очен', 3167),
 ('прост', 3116)]

In [10]:
# оставим только 5к слов
words_list = words_list[:5000]

# кол-во вхождений нам уже не нужно
words_list = [k[0] for k in words_list]

# для быстрого создания OHE полезно будет заранее пронумеровать каждое слово
# чтобы не ждать выполнения операции получения позиции в списке
words_ohe_positions = {words_list[i]: i for i in range(len(words_list))}

In [11]:
# списки под заголовки и тексты, которые тоже будут закодированы списками
titles = []
texts = []

# перебираем все строки
for row in df.itertuples():
    # сначала создаём шаблок с одними нулями
    title_ohe = [0] * len(words_list)
    for word in row.title:
        try:
            # если слово из заголовка присутствует в нашем словаре, увеличиваем счётчик на соответствующем месте
            title_ohe[words_ohe_positions[word]] += 1
        except:
            # если слово отсутствует, словарь выкинет исключение - в таком случае просто продолжаем цикл
            continue
    # делаем то же самое и для текста поста
    text_ohe = [0] * len(words_list)
    for word in row.text:
        try:
            text_ohe[words_ohe_positions[word]] += 1
        except:
            continue
    # добавляем получившуюся кодировку в списки заголовков и текстов
    titles.append(title_ohe)
    texts.append(text_ohe)

# для работы с Keras информацию лучше держать в Numpy
titles = np.array(titles)
texts = np.array(texts)

In [12]:
# проверим, правильно ли получилось

titles.shape, texts.shape

((7443, 5000), (7443, 5000))

In [13]:
# ответы тоже перегоним в Numpy

y = np.array(df['bad'])

In [14]:
# чтобы не заниматься копипастом 3 раза, сделаем функцию,
# которая разделяет массив в нужной пропорции
def train_val_test_split(x, val_frac=0.15, test_frac=0.15):
    x_train = x[:round((1 - val_frac - test_frac) * len(x))]
    x_val = x[round((1 - val_frac - test_frac) * len(x)):round((1 - test_frac) * len(x))]
    x_test = x[round((1 - test_frac) * len(x)):]
    return x_train, x_val, x_test


titles_train, titles_val, titles_test = train_val_test_split(titles)
texts_train, texts_val, texts_test = train_val_test_split(texts)
y_train, y_val, y_test = train_val_test_split(y)

In [15]:
# сначала 2 входа под заголовок и текст
title_input = tf.keras.layers.Input(shape=(5000,))
text_input = tf.keras.layers.Input(shape=(5000,))

# к каждому входу привязываем Dense-слой
title_dense_1 = tf.keras.layers.Dense(500, activation='relu')(title_input)
text_dense_1 = tf.keras.layers.Dense(500, activation='relu')(text_input)

# к ним привязываем батчнормы
title_bn_1 = tf.keras.layers.BatchNormalization()(title_dense_1)
text_bn_1 = tf.keras.layers.BatchNormalization()(text_dense_1)

# потом ещё Dense
text_dense_2 = tf.keras.layers.Dense(500, activation='relu')(text_bn_1)

# и ещё батчнормы
text_bn_2 = tf.keras.layers.BatchNormalization()(text_dense_2)

# склеиваем 2 слоя в один большой при помощи слоя Concatenate
add = tf.keras.layers.Add()([title_bn_1, text_bn_2])

# после этого докинем ещё пару слоёв с батчнормами
main_dense_1 = tf.keras.layers.Dense(300, activation='relu')(add)
main_bn_1 = tf.keras.layers.BatchNormalization()(main_dense_1)
drp1 = tf.keras.layers.Dropout(0.8)(main_bn_1)
main_dense_2 = tf.keras.layers.Dense(100, activation='relu')(drp1)
main_bn_2 = tf.keras.layers.BatchNormalization()(main_dense_2)
drp2 = tf.keras.layers.Dropout(0.8)(main_bn_2)

# выходной слой под задачу бинарной классификации
output = tf.keras.layers.Dense(1, activation='sigmoid')(drp2)

# теперь необходимо создать модель, указав в аргументах входные и выходные слои
# все остальные слои подтянутся автоматически, т.к. они уже связаны в единый граф
model = tf.keras.Model(inputs=[title_input, text_input], outputs=output)

In [16]:
# посмотрим, что получилось

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 5000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 500)       │  2,500,500 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 5000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 500)       │      2,000 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 500)       │  2,500,500 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 500)       │    250,500 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 500)       │      2,000 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 500)       │      2,000 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 500)       │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 300)       │    150,300 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300)       │      1,200 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 300)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 100)       │     30,100 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 100)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        101 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,439,601 (20.75 MB)

 Trainable params: 5,435,801 (20.74 MB)

 Non-trainable params: 3,800 (14.84 KB)

In [17]:
accuracy = tf.keras.metrics.binary_accuracy
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
auc = tf.keras.metrics.AUC()

def f1_metrics(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec) / (prec + rec + 1e-7))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[accuracy,
                       precision,
                       recall,
                       f1_metrics,
                       auc])

In [18]:
import os

os.mkdir('logs')

In [19]:
# создадим коллбэк

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/homework')

In [20]:
model.fit([titles_train, texts_train], y_train,  # данные на вход указываем в списке в нужном порядке
          validation_data=([titles_val, texts_val], y_val),
          batch_size=256,
          epochs=10,
          callbacks=[tb_callback])

Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - auc: 0.5940 - binary_accuracy: 0.5441 - f1_metrics: 0.3774 - loss: 1.1176 - precision: 0.2820 - recall: 0.6123 - val_auc: 0.7196 - val_binary_accuracy: 0.7637 - val_f1_metrics: 0.0000e+00 - val_loss: 0.5848 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - auc: 0.7328 - binary_accuracy: 0.6520 - f1_metrics: 0.4817 - loss: 0.7869 - precision: 0.3721 - recall: 0.7074 - val_auc: 0.7999 - val_binary_accuracy: 0.8093 - val_f1_metrics: 0.3220 - val_loss: 0.5265 - val_precision: 0.9322 - val_recall: 0.2083
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - auc: 0.8873 - binary_accuracy: 0.7871 - f1_metrics: 0.6193 - loss: 0.4956 - precision: 0.5287 - recall: 0.8377 - val_auc: 0.9052 - val_binary_accuracy: 0.8720 - val_f1_metrics: 0.6170 - val_loss: 0.4299 - val_precision: 0.9764 - val_recall: 0.4697
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - auc: 0.9540 - binary_accuracy: 0

In [21]:
import os

# Проверка существования папки
log_path = 'logs/homework'
if os.path.exists(log_path):
    print(f"\nСодержимое папки {log_path}:")
    for item in os.listdir(log_path):
        item_path = os.path.join(log_path, item)
        if os.path.isfile(item_path):
            size = os.path.getsize(item_path)
            print(f"Файл: {item} ({size} байт)")
        else:
            print(f"Папка: {item}")
else:
    print(f"Папка {log_path} не существует!")


Содержимое папки logs/homework:
Папка: train
Папка: validation


In [22]:
def check_logs_structure():
    log_path = 'logs'
    print("Полная структура папки logs:")
    
    for root, dirs, files in os.walk(log_path):
        level = root.replace(log_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            if file.endswith('.tfevents'):
                print(f"{subindent}Файл: {file}")
    
    # Проверим конкретно homework
    homework_path = 'logs/homework'
    if os.path.exists(homework_path):
        print(f"\nДетальное содержимое {homework_path}:")
        for item in os.listdir(homework_path):
            item_path = os.path.join(homework_path, item)
            if os.path.isdir(item_path):
                print(f"Папка: {item}")
                for subitem in os.listdir(item_path):
                    if subitem.startswith('events.out.tfevents'):
                        full_path = os.path.join(item_path, subitem)
                        size = os.path.getsize(full_path)
                        print(f"  → Файл: {subitem} ({size} байт)")
            else:
                if item.startswith('events.out.tfevents'):
                    size = os.path.getsize(item_path)
                    print(f"Файл: {item} ({size} байт)")

check_logs_structure()

Полная структура папки logs:
logs/
  homework/
    train/
    validation/

Детальное содержимое logs/homework:
Папка: train
  → Файл: events.out.tfevents.1758188222.MacBookPro.4856.0.v2 (20718 байт)
Папка: validation
  → Файл: events.out.tfevents.1758188225.MacBookPro.4856.1.v2 (9850 байт)


In [23]:
# После model.fit() добавьте:
print("Проверка создания файлов после обучения...")
for root, dirs, files in os.walk('logs'):
    for file in files:
        if 'tfevents' in file:
            full_path = os.path.join(root, file)
            size = os.path.getsize(full_path)
            print(f"Найден: {full_path} ({size} байт)")

Проверка создания файлов после обучения...
Найден: logs/homework/train/events.out.tfevents.1758188222.MacBookPro.4856.0.v2 (20718 байт)
Найден: logs/homework/validation/events.out.tfevents.1758188225.MacBookPro.4856.1.v2 (9850 байт)


In [24]:
import os
import subprocess

# Автоматически получаем полный путь
project_path = os.getcwd()
log_dir = os.path.join(project_path, 'logs', 'homework')

print(f"Запустите TensorBoard командой:")
print(f"tensorboard --logdir={log_dir}")

Запустите TensorBoard командой:
tensorboard --logdir=/Users/uliamalueva/Workshop/Tensorflow + Keras/logs/homework
